<h1><span style="background-color:blue; color:white; padding: 5px;">Notebook à executer lorsqu'il y a de nouvelles données disponible sur data.gouv.fr<br></h1>

In [ ]:
from functions2 import *

# Librairies pour la décompression de fichiers et système
import gzip
from io import BytesIO
import sys

# Librairies pour le scrapping
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Visualisation de données
import pandas as pd

# datas processing
import datas_processing

<h2><span style="background-color:grey; color:white; padding: 5px;">Récupération des données disponible et insertion dans la table<br></h2>

In [ ]:
# Récupération des codes communes , noms communes et code départements disponibles sur l'api 
url = "https://geo.api.gouv.fr/communes"
response = requests.get(url)
data_communes = response.json()
df_communes = pd.DataFrame(data_communes)
df_communes = df_communes.loc[:,["code","nom","codeDepartement"]]
df_communes = df_communes.drop_duplicates()
df_communes.columns= ["ID_COMMUNE","NAME_COMMUNE","ID_DEPT"]
liste_ID_COMMUNES = df_communes.ID_COMMUNE.to_list()

# URL de la page web
url = 'https://files.data.gouv.fr/geo-dvf/latest/csv/'

response = requests.get(url)
html_content = response.content

# Utilisez BeautifulSoup pour analyser le HTML
soup = BeautifulSoup(html_content, 'html.parser')

# Trouver toutes les balises <a>
a_tags = soup.find_all('a')
annees=[]
# Parcourir chaque balise <a> et extraire le texte ainsi que la date/heure
for a_tag in a_tags:
    name = a_tag.text.strip()
    if name!="../":
        annees.append(name)
print(annees)

url = f'https://files.data.gouv.fr/geo-dvf/latest/csv/{annee[-1]}'
response = requests.get(url)
html_content = response.content
soup = BeautifulSoup(html_content, 'html.parser')

# Trouver la balise 'a' qui contient les liens de téléchargement
csv_element = soup.find('pre').find('a', {'href': "full.csv.gz"})
csv_link = urljoin(url, csv_element['href'])

# Téléchargement des datas et conversion en data frame
csv_response = requests.get(csv_link)
if csv_response.status_code == 200:
    # Utilisation du buffer pour décompresser le fichier .gz
    try :
        with BytesIO(csv_response.content) as file_buffer:
            with gzip.GzipFile(fileobj=file_buffer, mode='rb') as gz_file:
                df = pd.read_csv(gz_file, low_memory=False)
    except Exception as e:
        print("Erreur dans la transformation des données en data frame :",e)

# Selection des données
df = datas_processing.select_datas(df)

# Gestion des valeurs manquantes :
df = datas_processing.nan_management(df)

# Adaptation des données du data frame au format de la base de données
df = datas_processing.format_data(df)

# Groupement des données sur une seule ligne par id_mutation (vente)
df = datas_processing.grouped_datas(df)

# Adaptation des données avec les variables et clés étrangères de la bdd
df = datas_processing.features_and_foreign_keys(df,liste_ID_COMMUNES)

#Suppression des lignes dupliquées
df = df.drop_duplicates()


engine = connection_with_sqlalchemy("datagouv")

#//////////////////////////////////////////////////////////////////////////////
#   Suppression de ventes identiques entre les nouvelles données récupérées
#                    et celles stockées en base de données
#//////////////////////////////////////////////////////////////////////////////
transaction = engine.begin()
try:
    # Récupération la date la plus ancienne dans les nouvelles données récupérées
    date_plus_ancienne = df['DATE_MUTATION'].min()
    print(date_plus_ancienne)
    # Requête pour supprimer les lignes avec une date supérieure ou égale à date_plus_ancienne
    query_delete = f"DELETE FROM VENTES WHERE DATE_MUTATION >= '{date_plus_ancienne}'"
    engine.execute(query_delete)

    # Validez la transaction
    transaction.commit()
    print("Transaction validée avec succès.")

except Exception as e:
    # En cas d'erreur, annulez la transaction
    print(f"Erreur : {e}")
    transaction.rollback()
    print("Transaction annulée.")


#//////////////////////////////////////////////////////////////////////////////
#                   Insertion de nouvelles données
#//////////////////////////////////////////////////////////////////////////////
transaction = engine.begin()
try:
    # Insère les données dans la table VENTES
    df.to_sql(name='VENTES', con=engine, if_exists='append', index=False)
    
    # Valide la transaction
    transaction.commit()
    print("Transaction validée avec succès.")

except Exception as e:
    # En cas d'erreur, annule la transaction
    print(f"Erreur : {e}")
    transaction.rollback()
    print("Transaction annulée.")

finally:
    # Libère les ressources de la connexion
    transaction.close()
    engine.dispose()

In [ ]:
# Récupération de la date de mise à jour du fichier
date_element = soup.find('pre').contents[-1].strip()
print(date_element)

<h2><span style="background-color:grey; color:white; padding: 5px;">Entraînement d'un modèle<br></h2>

In [ ]:
# Se baser sur les modèles créé dans mlflow et entraîner un modèle sur les mêmes paramètres
# Enregistrer dans mlflow, experiment : prod_{date_du_jour}
gestion_outliers =True
nb_ventes_mini = 10
tx_filtrage =[nb_ventes_mini,30]
region=''
type_de_bien =''
surface_terrain = True
nb_mois=15
param_grid = {
    'n_estimators': [100,150],
    'max_depth': [10],
    'learning_rate': [0.1],
    'min_child_weight': [1,3,5],
    'alpha' : [0.5], 
    'lambda' : [0.5]
}
cv=5
uri_tracking = "https://mlflowimmoappkevleg-737621d410d0.herokuapp.com/"
experiment_name = "En_Production"

# runs à effectuer all_datas 15months all_datas_surface_terrain 15months_surface_terrain)
run_name = f"{region}_{'Appartement_Maison' if len(type_de_bien)==0 else type_de_bien}"\
     f"_{nb_mois}months_GridSearch_GestionOutliers_{'oui' if gestion_outliers==True else 'non'}"\
     f"_{'' if surface_terrain==False else 'surface_terrain'}"

model_name = f"Production_XGB_{run_name}"

where_clause, query = construcion_requete(region,
                        type_de_bien,
                        nb_ventes_mini,
                        surface_terrain,
                        nb_mois)
print(query)

<h2><span style="background-color:grey; color:white; padding: 5px;">Créer un csv pour être utilisé dans streamlit<br></h2>

In [ ]:
# Pour éviter de trop requêter sur rds mais aussi avoir une execution rapide de l'application.
# Récupérer les données Region, département, communes, prix_M²/mois sur la dernière année ou 
# les deux dernières années suivant le modèle utilisé et par type de bien
df = loading_data("""

""")

<h2><span style="background-color:grey; color:white; padding: 5px;">Transformation du dataframe en csv et stockage dans AWS s3<br></h2>

In [1]:
import numpy as np
print(np.arange(50, 400, 50))

[ 50 100 150 200 250 300 350]


In [3]:
print([i / 10.0 for i in range(1, 11)])

[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
